In [13]:
import math
import yfinance as yf
import numpy as np
import pandas as pd
from datetime import date, timedelta, datetime
import time
import talib as ta

In [14]:
def get_longterm_stocklist():
  df = pd.read_csv('long_term_2022_list.csv')
  return df

In [15]:
def download_stock(tckr, start_date, end_date):
  start = start_date
  end = end_date
  symbol = tckr
  symbol = symbol+'.NS'
  stock_data = yf.download(symbol, start=start, end=end,rounding=True)
  if(len(stock_data)<1):
    stock_data = pd.DataFrame()
  return stock_data

In [16]:
def Gann_prediction(LTP):
  
  # find sq root of the ltp
  sq_rt_ltp = math.sqrt(LTP)
  #ceil the sq rt
  cieled_sq_rt = math.ceil(sq_rt_ltp)
  #get starting number for iteration
  lowest_num = cieled_sq_rt-2
  block = [lowest_num, lowest_num+1, lowest_num+2]
  
  for i in (block):
    centre_1 = math.pow(i+0.125, 2)
    centre_2 = math.pow(i+0.125*2, 2)
    centre_3 = math.pow(i+0.125*3, 2)
    centre_4 = math.pow(i+0.125*4, 2)
    centre_5 = math.pow(i+0.125*5, 2)
    centre_6 = math.pow(i+0.125*6, 2)
    centre_7 = math.pow(i+0.125*7, 2)
    centre_8 = math.pow(i+0.125*8, 2)
    if(i==block[0]):
      level_1 = [centre_1, centre_2, centre_3,centre_4,centre_5,centre_6,centre_7,centre_8]
    if(i==block[1]):
      level_2 = [centre_1, centre_2, centre_3,centre_4,centre_5,centre_6,centre_7,centre_8]
    if(i==block[2]):
      level_3 = [centre_1, centre_2, centre_3,centre_4,centre_5,centre_6,centre_7,centre_8]
  
  #merge list
  Gann_list = level_1 + level_2 + level_3


  insrt_loc = 0  
  for j in range(len(Gann_list)):
    if(Gann_list[j] < LTP < Gann_list[j+1]):
      insrt_loc = j+1
      Gann_list.insert(insrt_loc, LTP)
      break

  Resistance_1, Resistance_2, Resistance_3, Resistance_4 = (Gann_list[insrt_loc+2], Gann_list[insrt_loc+3],
                                                                       Gann_list[insrt_loc+4], Gann_list[insrt_loc+5])

  Support_1, Support_2, Support_3, Support_4 = (Gann_list[insrt_loc-2], Gann_list[insrt_loc-3],
                                                                       Gann_list[insrt_loc-4], Gann_list[insrt_loc-5])

  buy_above = Gann_list[insrt_loc+1]
  c1 = 0.99995
  sell_target_1, sell_target_2, sell_target_3, sell_target_4 =  (round(c1*Resistance_1,2), round(c1*Resistance_2,2), 
                                                                              round(1*Resistance_3,2), round(c1*Resistance_4,2))
  sl_long = Gann_list[insrt_loc-1]
  sell_below = Gann_list[insrt_loc-1]
  c2 = 1.00005
  buy_target_1, buy_target_2, buy_target_3, buy_target_4 = (round(c2*Support_1,2), round(c2*Support_2,2), round(c2*Support_3,2), 
                                                                        round(c2*Support_4,2))
  sl_short = Gann_list[insrt_loc+1]
  
  return (buy_above, sl_long, sell_target_1, sell_target_2, sell_target_3, sell_target_4,
          sell_below, sl_short, buy_target_1, buy_target_2, buy_target_3, buy_target_4)
  

In [17]:
def profit_percentage(prediction_list_df): # this function will use multiple levls of filtering
    
    stock_tckr = prediction_list_df['stock_tckr']
    buy_price =  prediction_list_df['buy_above'] #buying price for long trade
    sell_price = prediction_list_df['sell_below'] # selling price for short trade
    sl_long = prediction_list_df['sl_long']
    sl_short = prediction_list_df['sl_short']

    # Percentage profit with target 1 achieved for long trade
    pct_profit_long_tgt_1 = (prediction_list_df['sell_target_1'] - buy_price) * 100/ buy_price
    # Percentage profit with target 2 achieved for long trade
    pct_profit_long_tgt_2 = (prediction_list_df['sell_target_2'] - buy_price) * 100/ buy_price
    # Percentage profit with target 3 achieved for long trade
    pct_profit_long_tgt_3 = (prediction_list_df['sell_target_3'] - buy_price) * 100/ buy_price
    # Percentage profit with target 4 achieved for long trade
    pct_profit_long_tgt_4 = (prediction_list_df['sell_target_4'] - buy_price) * 100/ buy_price

    # Percentage profit with target 1 achieved for short trade
    pct_profit_short_tgt_1 = (sell_price - prediction_list_df['buy_target_1']) *100/ prediction_list_df['buy_target_1']
    # Percentage profit with target 2 achieved for short trade
    pct_profit_short_tgt_2 = (sell_price - prediction_list_df['buy_target_2']) *100/ prediction_list_df['buy_target_2']
    # Percentage profit with target 3 achieved for short trade
    pct_profit_short_tgt_3 = (sell_price - prediction_list_df['buy_target_3']) *100/ prediction_list_df['buy_target_3']
    # Percentage profit with target 4 achieved for short trade
    pct_profit_short_tgt_4 = (sell_price - prediction_list_df['buy_target_4']) *100/ prediction_list_df['buy_target_4']
    
    # check stop loss vs pct profit
    # Risk reward percentage: long trade vs stop loss
    loss_pct_against_long_trade = (buy_price - sl_long) * 100/ buy_price
    # Risk reward percentage: short trade vs stop loss
    loss_pct_against_short_trade = (sl_short - sell_price) * 100/ sl_short

    return (stock_tckr, pct_profit_long_tgt_1, pct_profit_long_tgt_2, pct_profit_long_tgt_3, pct_profit_long_tgt_4,
            pct_profit_short_tgt_1, pct_profit_short_tgt_2, pct_profit_short_tgt_3, pct_profit_short_tgt_4,
            loss_pct_against_long_trade, loss_pct_against_short_trade)

In [18]:
def stock_filtering(long_profit, long_loss, short_profit, short_loss):
    if((long_profit>(2*long_loss) or short_profit>(2*short_loss)) and (long_profit>2 or short_profit>2)):
        return "good_stock"
    else:
        return "remove_stock"


In [19]:
# Main 
longterm_stock_list = get_longterm_stocklist()
prediction_list =[]
risk_rewards = []
#current_date = date.today().strftime("%Y-%m-%d")
start_date = pd.Timestamp('today') - timedelta(days=1)# previous 1 day from today
start_date = start_date.strftime("%Y-%m-%d")

current_date = pd.Timestamp('today') + timedelta(days=1) # this adjustment is required because python does not include last date of the range.
current_date = current_date.strftime("%Y-%m-%d")


In [20]:
# Gann prediction algorithm
for i in range(len(longterm_stock_list)-1):
#for i in range(0,10): # test with 3 stocks
  longterm_stock_tckr = longterm_stock_list['stock_tckr'][i] # get ticker symbol
  longterm_stock_data = download_stock(longterm_stock_tckr, start_date ,current_date) # download data from yfinance 
  if(len(longterm_stock_data)>0 and not(pd.isnull(longterm_stock_data.iloc[0,3]))):
    current_day_close = round(longterm_stock_data.values[0][3],2)

    (buy_above, sl_long, sell_target_1, sell_target_2, sell_target_3, sell_target_4,
           sell_below, sl_short, buy_target_1, buy_target_2, buy_target_3, buy_target_4)= Gann_prediction(current_day_close)
          
    prediction_list.append([longterm_stock_tckr,buy_above, sl_long, sell_target_1, sell_target_2, sell_target_3, sell_target_4,
           sell_below, sl_short, buy_target_1, buy_target_2, buy_target_3, buy_target_4])

prediction_list_df = pd.DataFrame(prediction_list, columns=['stock_tckr','buy_above', 'sl_long', 'sell_target_1', 'sell_target_2', 
        'sell_target_3', 'sell_target_4', 'sell_below', 'sl_short', 'buy_target_1', 'buy_target_2', 'buy_target_3', 'buy_target_4'])



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [21]:
prediction_list_df

,stock_tckr,buy_above,sl_long,sell_target_1,sell_target_2,sell_target_3,sell_target_4,sell_below,sl_short,buy_target_1,buy_target_2,buy_target_3,buy_target_4
0,RELIANCE,2665.140625,2652.250000,2677.93,2690.88,2704.00,2716.88,2652.250000,2665.140625,2639.52,2626.69,2613.90,2601.13
1,TCS,3766.890625,3751.562500,3782.06,3797.45,3813.06,3828.32,3751.562500,3766.890625,3736.45,3721.19,3705.95,3690.75
2,HINDUNILVR,2081.640625,2070.250000,2092.96,2104.41,2116.00,2127.41,2070.250000,2081.640625,2058.99,2047.66,2036.37,2025.10
3,INFY,1914.062500,1903.140625,1924.92,1935.90,1947.02,1957.96,1903.140625,1914.062500,1892.34,1881.48,1870.66,1859.86
4,HDFC,2462.640625,2450.250000,2474.94,2487.39,2500.00,2512.39,2450.250000,2462.640625,2438.01,2425.68,2413.39,2401.12
5,BHARTIARTL,763.140625,756.250000,770.02,776.98,784.00,790.98,756.250000,763.140625,749.43,742.60,735.80,729.04
6,ITC,256.000000,252.015625,260.00,264.05,268.14,272.24,252.015625,256.000000,248.07,244.15,240.26,236.40
7,ASIANPAINT,3122.015625,3108.062500,3135.84,3149.86,3164.06,3177.98,3108.062500,3122.015625,3094.30,3080.40,3066.54,3052.72
8,NESTLEIND,17589.390625,17556.250000,17621.68,17654.88,17689.00,17721.38,17556.250000,17589.390625,17524.02,17490.94,17457.89,17424.87
9,HCLTECH,1173.062500,1164.515625,1181.58,1190.19,1198.89,1207.50,1164.515625,1173.062500,1156.06,1147.57,1139.12,1130.70


In [22]:
risk_rewards =[]

In [23]:
#filtering low profit stocks
#for i in range(0,100): # test with 3 stocks
for i in range(len(prediction_list_df)):
  (stock_tckr, pct_profit_long_tgt_1, pct_profit_long_tgt_2, pct_profit_long_tgt_3, pct_profit_long_tgt_4,
  pct_profit_short_tgt_1, pct_profit_short_tgt_2, pct_profit_short_tgt_3, pct_profit_short_tgt_4,
  loss_pct_against_long_trade, loss_pct_against_short_trade) = profit_percentage(prediction_list_df.iloc[i])

  risk_rewards.append([stock_tckr, pct_profit_long_tgt_1, pct_profit_long_tgt_2, pct_profit_long_tgt_3, pct_profit_long_tgt_4,
  pct_profit_short_tgt_1, pct_profit_short_tgt_2, pct_profit_short_tgt_3, pct_profit_short_tgt_4,
  loss_pct_against_long_trade, loss_pct_against_short_trade])

risk_rewards_df = pd.DataFrame(risk_rewards, columns = ['stock_tckr', 'pct_profit_long_tgt_1', 'pct_profit_long_tgt_2', 'pct_profit_long_tgt_3',
 'pct_profit_long_tgt_4', 'pct_profit_short_tgt_1', 'pct_profit_short_tgt_2', 'pct_profit_short_tgt_3', 'pct_profit_short_tgt_4', 
 'loss_pct_against_long_trade', 'loss_pct_against_short_trade'])

# Merge both data frames, Gann predictions and risk rewards for evaluation
reduced_stocks_df = pd.merge(prediction_list_df, risk_rewards_df,  on = 'stock_tckr')


In [24]:
reduced_stocks_df.to_csv('longterm_predictions_for_2022Q1.csv')